In [1]:
!date

Sun May 18 09:07:42 PM EDT 2025


In [2]:
pwd

'/projects/sccn/andromeda1/aglinska/BC-ABCD-denoise/Code'

In [3]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ants
import pickle
from tqdm import tqdm

In [4]:
def load_pickle(fn):
    if os.path.exists(fn):
        with open(fn, 'rb') as file:
            loaded_dict = pickle.load(file)
    return loaded_dict

In [5]:
def correlate_columns(arr1, arr2):
    """
    Computes the Pearson correlation between corresponding columns of two matrices.
    
    Parameters:
    arr1 (np.ndarray): First matrix of shape (370, 1000)
    arr2 (np.ndarray): Second matrix of shape (370, 1000)
    
    Returns:
    np.ndarray: 1D array of correlations for each column (size 1000)
    """
    # Ensure input arrays are numpy arrays
    arr1 = np.asarray(arr1)
    arr2 = np.asarray(arr2)
    
    # Subtract the mean of each column (normalize)
    arr1_centered = arr1 - np.mean(arr1, axis=0)
    arr2_centered = arr2 - np.mean(arr2, axis=0)
    
    # Compute the numerator (covariance)
    numerator = np.sum(arr1_centered * arr2_centered, axis=0)
    
    # Compute the denominator (product of standard deviations)
    denominator = np.sqrt(np.sum(arr1_centered**2, axis=0) * np.sum(arr2_centered**2, axis=0))
    
    # Compute the Pearson correlation for each column
    correlation = numerator / denominator
    
    return correlation

In [7]:
def get_regs(events_fn):
    from nilearn.glm.first_level import make_first_level_design_matrix

    events = pd.read_csv(events_fn,delimiter='\t')

    t_r = 2.0 
    #n_scans = epi.shape[-1]
    n_scans = 156
    frame_times = (np.arange(n_scans) * t_r)

    X1 = make_first_level_design_matrix(frame_times,events,drift_model="polynomial",drift_order=3,hrf_model="SPM") #

    face_reg = X1[['face']].values.sum(axis=1)
    place_reg = X1[['body', 'house', 'object', 'scene', 'scramble']].values.sum(axis=1)
    
    return face_reg,place_reg

events_fn_temp = '../Data/StudyForrest/events/{sub}_ses-localizer_task-objectcategories_run-{r}_events.tsv'
face_reg,place_reg = get_regs(events_fn_temp.format(sub='sub-01',r=1))

In [8]:
subs = [sub for sub in os.listdir('../Data/StudyForrest/fmriprep/') if sub.startswith('sub-')]
subs.sort()
print(len(subs))

14


In [9]:
analysis_dir = '../Data/StudyForrest/ensembles_last_CVAE/'
analysis_name = 'fixed-seed-00-YuOrig-00-DataloaderFix'
#analysis_name = 'DeepCor-Forrest-face-v5'

In [22]:
def get_data(s,sub,r):
    allOutputs = []
    for rep in range(20):
        fn_pickle = os.path.join(analysis_dir,analysis_name,f'outputs_S{s}_R{r}_rep_{rep}.pickle')
        if os.path.exists(fn_pickle):
            outputs = load_pickle(fn_pickle)
            allOutputs.append(outputs)

    allOutputs_signal = np.array([val['signal'] for val in allOutputs])
    allOutputs_ffa = allOutputs[0]['ffa']
    allOutputs_ffa_compcorr = allOutputs[0]['ffa_compcorr']

    allOutputs_signal_avg = np.average(allOutputs_signal,axis=0)

    im_signal = ants.image_read(os.path.join(analysis_dir,analysis_name,f'signal_S{s}_R{r}_avg.nii.gz'))
    im_subFFA = ants.image_read(f'../Data/StudyForrest/ROIs/rFFA_final_mask_{sub}_bin.nii.gz')

    fn_preproc = '../Data/StudyForrest/fmriprep/{sub}/ses-localizer/func/{sub}_ses-localizer_task-objectcategories_run-{r}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
    im_preproc = ants.image_read(fn_preproc.format(sub=sub,r=r))

    fn_compcor = '../Data/StudyForrest/fmriprep/{sub}/ses-localizer/func/{sub}_ses-localizer_task-objectcategories_run-{r}_bold_space-MNI152NLin2009cAsym_COMPCOR-RONI.nii'
    im_compcor = ants.image_read(fn_compcor.format(sub=sub,r=r))

    vals_subFFA_signal = im_signal.numpy()[im_subFFA.numpy()==1]
    vals_subFFA_preproc = im_preproc.numpy()[im_subFFA.numpy()==1]
    vals_subFFA_compcor = im_compcor.numpy()[im_subFFA.numpy()==1]

    v_idx = vals_subFFA_signal.std(axis=1)>1e-3

    vals_subFFA_signal = vals_subFFA_signal[v_idx]
    vals_subFFA_preproc = vals_subFFA_preproc[v_idx]
    vals_subFFA_compcor = vals_subFFA_compcor[v_idx]

    #face_reg = outputs['face_reg']

    #nv = vals_subFFA_signal.shape[0]
    nv = vals_subFFA_preproc.shape[0]

    r_im_signal = correlate_columns(vals_subFFA_signal.transpose(),np.array([face_reg for _ in range(nv)]).transpose())
    r_im_preproc = correlate_columns(vals_subFFA_preproc.transpose(),np.array([face_reg for _ in range(nv)]).transpose())
    r_im_compcor = correlate_columns(vals_subFFA_compcor.transpose(),np.array([face_reg for _ in range(nv)]).transpose())
    (r_im_signal.mean().round(4),r_im_preproc.mean().round(4),r_im_compcor.mean().round(4))

    r_outputs_signal = correlate_columns(allOutputs_signal_avg[v_idx,:].transpose(),np.array([face_reg for _ in range(nv)]).transpose())
    r_outputs_preproc = correlate_columns(allOutputs_ffa[v_idx,:].transpose(),np.array([face_reg for _ in range(nv)]).transpose())
    r_outputs_compcor = correlate_columns(allOutputs_ffa_compcorr[v_idx,:].transpose(),np.array([face_reg for _ in range(nv)]).transpose())

    (r_outputs_signal.mean().round(4),r_outputs_preproc.mean().round(4),r_outputs_compcor.mean().round(4))
    
    #res = [r_im_signal,r_im_preproc,r_im_compcor,r_outputs_signal,r_outputs_preproc,r_outputs_compcor]
    res = [r_im_signal.mean(),r_im_preproc.mean(),r_im_compcor.mean(),r_outputs_signal.mean(),r_outputs_preproc.mean(),r_outputs_compcor.mean()]
    
    return res

In [23]:
result = np.zeros((14,4,6))

In [24]:
for s in tqdm(range(14)):
    sub = subs[s]
    for r in [1,2,3,4]:
        result[s,r-1,:] = get_data(s,sub,r)

100%|██████████| 14/14 [01:31<00:00,  6.54s/it]


In [25]:
result_avg = result.mean(axis=0).mean(axis=0)

In [26]:
result_avg

array([0.21009251, 0.15116312, 0.16787899, 0.21007557, 0.15116311,
       0.16897971])

In [31]:
print(result_avg[0:3])
print(result_avg[3::])

[0.21009251 0.15116312 0.16787899]
[0.21007557 0.15116311 0.16897971]


In [32]:
np.corrcoef(result[:,:,0].flatten(),result[:,:,3].flatten())[0,1]

0.999999706915353

In [33]:
np.corrcoef(result[:,:,1].flatten(),result[:,:,4].flatten())[0,1]

0.9999999999999792

In [34]:
np.corrcoef(result[:,:,2].flatten(),result[:,:,5].flatten())[0,1]

0.9829791338106583

In [20]:
#outputs.keys()